In [2]:
#!pip install langchain_cohere
#!pip install --force-reinstall --no-cache-dir langchain-cohere

In [3]:
class Settings:
    from dotenv import load_dotenv
    import os
    load_dotenv("api_keys.env")
    cohere_api_key  = os.getenv("COHERE_API_KEY")

def get_settings():
    return Settings()

In [4]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage
from langchain.memory import ChatMessageHistory

# 設定の読み込み（APIキーなど）
settings = get_settings()

# Cohereクライアントの初期化
llm = ChatCohere(cohere_api_key=settings.cohere_api_key)

# プロンプトテンプレート（Chat形式）
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは親切なAIチューターです。以下の会話履歴に基づいて、日本語で1つ質問してください。トピック: {topic}"),
    ("human", "{input}")
])

# チェーンの構築
chain = prompt | llm

# 会話履歴を管理するための関数（ここでは単純な辞書を使う）
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# RunnableWithMessageHistoryでセッション管理付きに
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: get_session_history(session_id),
    input_messages_key="input",
    history_messages_key="history"
)

# 実行（session_id付きで）
response = chain_with_history.invoke(
    {"input": "寿司が好きです。", "topic": "食文化"},
    config={"configurable": {"session_id": "user-session-001"}}
)

print(response.content)


寿司がお好きとのことですが、**どんな種類の寿司が一番お好みですか？**


In [5]:
from IPython.display import display, HTML
import ipywidgets as widgets

# セッションIDを決めておく（毎回同じなら履歴が続く）
session_id = "user-session-001"

# テキスト入力と出力表示の準備
input_box = widgets.Text(placeholder="あなたの発言を入力してください", description="👤:")
output_area = widgets.Output()

# 入力イベント処理関数
def on_input_submit(sender):
    user_input = input_box.value
    input_box.value = ""  # 入力欄をクリア

    with output_area:
        print(f"👤: {user_input}")
        response = chain_with_history.invoke(
            {"input": user_input, "topic": "食文化"},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"🤖: {response.content}")

# 入力ボックスにイベントハンドラを登録
input_box.on_submit(on_input_submit)

# 表示
display(HTML("<h3>🗣️ AIと会話してみましょう</h3>"))
display(input_box, output_area)


ModuleNotFoundError: No module named 'ipywidgets'